In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import string
import re
from pickle import dump
import unicodedata as ud
from numpy import array
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs
 
# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	for pair in lines:
		clean_pair = list()
		for line in pair:
			# tokenize on white space
			line = line.split()
			# remove punctuation from each token
			#line = [c for c in line if not ud.category(c).startswith('P')]
			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned) 
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)
 
# load dataset
filename = 'output.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
# save clean pairs to file
save_clean_data(clean_pairs, 'msa-to-darija.pkl')
# spot check
for i in range(20):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

Saved: msa-to-darija.pkl
[إنها في أخر القاعة سوف آتي لك ببعض منها الآن إذا أردت أي شيئا آخر فقط أعلمني] => [كاين في اللاخر ديال هاد القاعة. انجيب ليك شويا دابا. و إلا حتاجيتي شي حاجا اخرى، قولها ليا.]
[هل تقومون بعمل تعديلات] => [واش كا دير التعديلات؟]
[نريد مائدة بجانب النافذة] => [بغينا ناخدو طابلة حدا الشرجم.]
[هناك أمام بيانات السائح تماما] => [راه تما، مقابل مكتب استعلامات السياح بالضبط.]
[لم اسمع بهذا العنوان من قبل بالقرب من هنا] => [ما عمري سمعت هاد العنوان هنايا.]
[استمر في السير في هذا الطريق حتى تجد صيدلية] => [سير نيشان حتا تشوف صيدلية.]
[ما هو أحدث لون هذا الموسم] => [شنو هو اجدد لون هاد الموسم؟]
[في حالتي غالبا ما يكون من أجل العمل ونادرا ما يكون للاستمتاع] => [في الحالة ديالي، راه غالبا على الخدمة، قليل للمتعة.]
[سأقيم لمدة يومين] => [غادي نبقا يومين.]
[أريد تمويج متقارب بشعري] => [بغيت بيرمانونت مجهد.]
[أريد حجز غرفتين مزدوجتين] => [بغيت نحجز جوج غرف ثنائية.]
[لا أريد إزالتها] => [ما بغيتهوش منزوع.]
[هل هذه الحافلة تتوقف في شارع أستونر] => [واش كا يحبس هاد الطوبيس في شا

In [ ]:

from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle
 
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)
 
# load dataset
raw_dataset = load_clean_sentences('msa-to-darija.pkl')
 
# reduce dataset size
n_sentences = 8000
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:6000], dataset[6000:]
# save
save_clean_data(dataset, 'msa-to-darija-both.pkl')
save_clean_data(train, 'msa-to-darija-train.pkl')
save_clean_data(test, 'msa-to-darija-test.pkl')

Saved: msa-to-darija-both.pkl
Saved: msa-to-darija-train.pkl
Saved: msa-to-darija-test.pkl


In [ ]:
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
# load datasets
dataset = load_clean_sentences('msa-to-darija-both.pkl')
train = load_clean_sentences('msa-to-darija-train.pkl')
test = load_clean_sentences('msa-to-darija-test.pkl')

In [ ]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [ ]:

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

In [ ]:
import os, sys

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('msa Vocabulary Size: %d' % eng_vocab_size)
print('msa Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('darija Vocabulary Size: %d' % ger_vocab_size)
print('darija Max Length: %d' % (ger_length))

Using TensorFlow backend.


msa Vocabulary Size: 10614
msa Max Length: 50
darija Vocabulary Size: 12602
darija Max Length: 50


In [ ]:

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

In [ ]:
# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = np.array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

In [ ]:

# prepare training data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)

In [ ]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model
 
# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)
# fit model
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 256)           3226112   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 50, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 10614)         2727798   
Total params: 7,004,534
Trainable params: 7,004,534
Non-trainable params: 0
_________________________________________________________________
None



Tra

In [ ]:
# load datasets
dataset = load_clean_sentences('msa-to-darija-both.pkl')
train = load_clean_sentences('msa-to-darija-train.pkl')
test = load_clean_sentences('msa-to-darija-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])

In [ ]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
# load model
model = load_model('model.h5')

In [ ]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [ ]:
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

In [ ]:
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
 
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)
 
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X
 
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)
 
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
 
# load datasets
dataset = load_clean_sentences('msa-to-darija-both.pkl')
train = load_clean_sentences('msa-to-darija-train.pkl')
test = load_clean_sentences('msa-to-darija-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
 
# load model
model = load_model('model.h5')
# test on some training sequences
print('train')
evaluate_model(model, eng_tokenizer, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, eng_tokenizer, testX, test)

train
src=[شي وجبة خفيفة، عافاك.], target=[وجبة خفيفة من فضلك], predicted=[هل أن]
src=[سمحلي، بغيت ن آنيلي الريزيرفاسيو ديالي.], target=[أنا آسف ولكنني أريد أن ألغي الحجز], predicted=[هل هل أن]
src=[التلج كايرجع الطريق كاتزلق.], target=[الثلج يجعل الطريق زلق], predicted=[هل هل أن]
src=[يالاه، جري.], target=[هيا أسرع], predicted=[هل هل]
src=[من إينا سكة كا يخرج التران لي غادي ل ميلانو؟], target=[أي رصيف يغادر منه القطار المتجهة إلى ميلان], predicted=[هل هل أن أن]
src=[علاش كاتفيق بكري كل صباح؟], target=[لماذا تستيقظ مبكرا كل صباح], predicted=[هل هل أن]
src=[غادي ناخد واحد احمر و واحد ابيض.], target=[سآخذ واحدة حمراء وواحدة بيضاء], predicted=[هل هل أن]
src=[قطعو من لا كارط ديالي، عفاك. ماستر كارط.], target=[اخصم تكلفتها من بطاقة ائتماني من فضلك ونوعها ماستر كارد], predicted=[هل هل أن أن]
src=[بالصح.], target=[حسنا], predicted=[هل هل]
src=[باقي عاقل على الشفار؟], target=[هل تتذكر ملامح السارق], predicted=[هل هل]


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-1: 0.037716
BLEU-2: 0.105221
BLEU-3: 0.158611
BLEU-4: 0.175748
test
src=[خاصني نبدل معا لفول رقم مياو تلاتة و عشرين.], target=[أتصل على التردد الصوتي واحد اثنان ثلاثة], predicted=[هل هل أن أن أن]
src=[فين يمكن ليا ناخد الطوبيس للمحطة الطرقية دالمدينة؟], target=[أين يمكن أن اركب الحافلة إلى موقف المدينة العمومي], predicted=[هل هل أن أن]
src=[واش غادي نوصلو في الوقت؟], target=[هل سنصل في الموعد], predicted=[هل هل أن]
src=[عادة هو خمسة وتلاتين دولار بلحاق اليوم كاين تخفيض ديال خمسة وعشرين فالمية إذن هو غير ستة وعشرين دولار وخمسة وعشرين سنت.], target=[إن ثمنها في المعتاد خمسة وثلاثون دولارا أما اليوم فثمة تخفيض قدره خمسة وعشرون في المائة ليتوقف ثمنها عند ستة وعشرين دولارا وخمسة وعشرين سنتا], predicted=[هل هل أن أن أن أن]
src=[راه كاين على اليمين.], target=[إنه هناك على الجانب الأيمن], predicted=[هل هل]
src=[لسوء الحظ، الغرف الفردية كلها محجوزة. بلحاق باقي لينا شي غرف مزدوجة.], target=[لسوء الحظ كل الغرف المفردة محجوزة ولكن لا يزال لدينا بضعة غرف مزدوجة], predicted=[هل هل أن أن أن]
src